In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import os
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.metrics import MeanIoU


from segmentation_models import Unet
from segmentation_models.metrics import iou_score

from sklearn.metrics import confusion_matrix
import cv2 as cv




In [ ]:
## import paths
path_train_img ='/.../training/images/images/'
path_train_mask='/.../training/mask/mask/'

path_val_img ='/.../validation/images/images/'
path_val_mask = '/.../validation/mask/mask/'

##test Images
image_train = cv.imread(path_train_img +"1.png")
mask_train = cv.imread(path_train_mask +"1.png")

img_color= np.array(image_train)
mask=np.array(mask_train)
mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)

len_training= len(os.listdir(path_train_img))
print("len_train",len_training)
len_validation= len(os.listdir(path_val_img))
print("len val",len_validation)
print("shape color img",img_color.shape)
print ("shape mask:",mask.shape)

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(221)
plt.title("cropped Imgage by RetinaNet Network")
plt.imshow(img_color)

plt.subplot(222)
plt.title("Ground thrut mask")
plt.imshow(mask, cmap="gray")



In [ ]:
##segmentation_preprocessing
from segmentation import SegmentationGenerator

num_classes=5
batch_size=8
size= 512
print(size)

step_per_epoch_ = len_training//batch_size
step_validation= len_validation//batch_size

print(step_validation)

train_generator= SegmentationGenerator(path_train_img,
                                      path_train_mask,
                                      num_classes=num_classes,
                                      rescale=1./255,
                                      target_size=(size,size),
                                      batch_size=batch_size,
                                      shuffle=True)

val_generator= SegmentationGenerator(path_val_img,
                                     path_val_mask,
                                     num_classes=num_classes,
                                     rescale=1./255,
                                     target_size=(size,size),
                                     batch_size=batch_size)


In [ ]:
def mapping_lava_flow_unet(pretrained_weights=None, input_size=(size,size,3)):
    
    inputs = Input(input_size)
    conv1 = Conv2D(16,3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(16,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2,2))(conv1)
    
    conv2 = Conv2D(32,3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(32,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2,2))(conv2)
    
    conv3 = Conv2D(64,3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(64,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2,2))(conv3)
    
    conv4 = Conv2D(128,3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(128,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    pool4 = MaxPooling2D(pool_size=(2,2))(conv4)
    
    conv5 = Conv2D(256,3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(256,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    pool5 = MaxPooling2D(pool_size=(2,2))(conv5)
    
    conv6 = Conv2D(512,3, activation='relu', padding='same', kernel_initializer='he_normal')(pool5)
    conv6 = Conv2D(512,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)
    drop6 = Dropout(0.5)(conv6)
    
    '''DECODER'''
    
    up7= Conv2DTranspose(256,3, strides=(2, 2), padding='same')(drop6)
    merge7= concatenate([conv5,up7])
    conv7= Conv2D(256,3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7= Conv2D(256,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8= Conv2DTranspose(128,3,  strides=(2, 2),padding='same',)(conv7)
    merge8= concatenate([conv4,up8],axis=3)
    conv8= Conv2D(128,3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8= Conv2D(128,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)
    
    up9= Conv2DTranspose(64,3,  strides=(2, 2), padding='same', )(conv8)
    merge9= concatenate([conv3,up9],axis=3)
    conv9= Conv2D(64,3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9= Conv2D(64,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    
    up10= Conv2DTranspose(32,3,  strides=(2, 2), padding='same')(conv9)
    merge10= concatenate([conv2,up10],axis=3)
    conv10= Conv2D(32,3, activation='relu', padding='same', kernel_initializer='he_normal')(merge10)
    conv10= Conv2D(32,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv10)
    
    up11= Conv2DTranspose(16,3,  strides=(2, 2), padding='same')(conv10)
    merge11= concatenate([conv1,up11],axis=3)
    conv11= Conv2D(16,3, activation='relu', padding='same', kernel_initializer='he_normal')(merge11)
    conv11= Conv2D(16,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv11)
    conv11= Conv2D(16,3, activation='relu', padding='same', kernel_initializer='he_normal')(conv11)
    
    classify= Conv2D(5,1, activation='softmax')(conv11)

    model = Model(inputs=[inputs], outputs=[classify])   
    return model

In [ ]:
stromboli_unet = mapping_lava_flow_unet()
stromboli_unet.summary()

In [ ]:
stop= tf.keras.callbacks.EarlyStopping(monitor='loss',
                                       patience=10)

stromboli_unet.compile(optimizer=tf.optimizers.Adam(learning_rate=0.0001),
                          loss='categorical_crossentropy',
                          metrics=['accuracy',iou_score])


In [ ]:
history = stromboli_unet.fit(train_generator,
                                validation_data=val_generator,
                                steps_per_epoch=step_per_epoch_,
                                validation_steps=step_validation,
                                callbacks=[stop],
                                epochs=2)

In [ ]:
plt.figure(figsize=(14,10))

plt.subplot(221)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epochs')
plt.ylabel('Loss function')
plt.legend(['training','validation'])
plt.title('Loss function U-Net model Stromboli')
plt.grid()


plt.subplot(222)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend(['training','validation'])
plt.title('Accuracy U-Net model Stromboli')
plt.grid()

In [ ]:
from tensorflow.keras.models import save_model
#stromboli_unet.save('D:/unet_traided_model.h5')

In [ ]:
from keras.models import load_model
#stromboli_unet= load_model('D:/unet_traided_model.h5',custom_objects= {'iou_score': iou_score})

In [ ]:
val_loss, val_acc, val_iou= stromboli_unet.evaluate(val_generator,
                                                 steps=int(len(val_generator)/4))
print('extactidu es:', format(100*val_acc,'.2f'),'%')
print('IOU es', format(100*val_iou,'.2f'),'%')

In [ ]:
# Function to calculate IoU for each image
def calculate_iou_per_image(y_true, y_pred, num_classes=5):
    y_true = np.argmax(y_true, axis=-1).flatten()
    y_pred = np.argmax(y_pred, axis=-1).flatten()

    iou_metric = MeanIoU(num_classes=num_classes)
    iou_metric.update_state(y_true, y_pred)
    return iou_metric.result().numpy()

iou_scores = []


for i, (x_val, y_true) in enumerate(val_generator):
    y_pred = stromboli_unet.predict(x_val)
    iou_score = calculate_iou_per_image(y_true, y_pred)
    iou_scores.append(iou_score)
    print(f"Image {i+1} - IoU: {iou_score:.4f}")

    if i >= len(val_generator) - 1:
        break

# Calculate the mean IoU score
mean_iou = np.mean(iou_scores)

plt.figure(figsize=(10, 6))
plt.plot(iou_scores, marker='o', linestyle='-', color='b', label='IoU per Image')
plt.axhline(y=mean_iou, color='r', linestyle='--', label=f'Mean IoU: {mean_iou:.4f}')
plt.title('Mean IoU Score for 250 Batches')
plt.xlabel('Image Index')
plt.ylabel('IoU Score')
plt.grid(True)
plt.legend()
plt.show()
